# Cell Segmentation with Watershed

In this notebook, we will segment cells from a microscopy image using the watershed algorithm. We aim to partition the image into meaningful regions representing individual cells.

## Step 1: Load the Image
We will first load the image that is saved in the `data` directory.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import io

# Load the image
image = io.imread('../data/membrane2d.png', as_gray=True)

# Display the original image
plt.imshow(image, cmap='gray')
plt.title('Original Image')
plt.axis('off')
plt.show()

## Step 2: Enhance Contrast
Enhancing the contrast of the image can help improve the segmentation results.

In [ ]:
from skimage import filters, morphology

# Enhance the contrast
image_enhanced = filters.rank.enhance_contrast(image, morphology.disk(3))

# Display the enhanced image
plt.imshow(image_enhanced, cmap='gray')
plt.title('Enhanced Contrast Image')
plt.axis('off')
plt.show()

## Step 3: Thresholding
We now apply a threshold to create a binary image that will help in identifying regions for segmentation.

In [ ]:
# Apply Otsu's threshold
binary = image_enhanced > filters.threshold_otsu(image_enhanced)

# Display the binary image
plt.imshow(binary, cmap='gray')
plt.title('Binary Image')
plt.axis('off')
plt.show()

## Step 4: Distance Transform
Next, we compute the distance transform to identify the background and prepare for the watershed algorithm.

In [ ]:
from scipy import ndimage as ndi

# Compute the distance transform
distance = ndi.distance_transform_edt(binary)

# Display the distance transform
plt.imshow(distance, cmap='magma')
plt.title('Distance Transform')
plt.axis('off')
plt.show()

## Step 5: Apply Watershed
We now apply the watershed algorithm to segment the cells based on the distance map.

In [ ]:
from skimage import feature, segmentation, measure

# Find local maxima
local_maxi = feature.peak_local_max(distance, indices=False, labels=binary)

# Generate markers
markers = ndi.label(local_maxi)[0]

# Apply watershed
labels = segmentation.watershed(-distance, markers, mask=binary)

# Display the segmented image
plt.imshow(labels, cmap='nipy_spectral')
plt.title('Watershed Segmentation')
plt.axis('off')
plt.show()

## Step 6: Measure Cell Areas
Lastly, we measure the area of segmented cell regions and calculate the average cell area.

In [ ]:
# Measure regions and calculate areas
props = measure.regionprops(labels)
areas = [prop.area for prop in props]
average_area = np.mean(areas)

print(f"Average cell area: {average_area:.2f} pixels")